In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

Here, we look at current-based synapse described by the differential equation
$$\tau\frac{dI(t)}{dt}=-I(t)+h\cdot s(t),$$
where $\tau$ is the intrinsic synaptic time constant, $h$ is the synaptic efficacy and $s(t)$ represents the incoming spikes 
$$s(t)=\sum_{i}\delta(t-t_{i})$$
The efficacy $h$ can be positive in case of an excitatory or negative in case of inhibitory synapse. Directly after a spike arrives, the current will make a jump by $h$ and than decay back to $0$ due to the leek term. 

In the following, we assume that $s(t)$ can be described by Poisson process with rate $\lambda$, i.e. the interspike interval (ISI) are drawn from an exponential distribution 
$$\text{ISI(}t)=\lambda e^{-\lambda t}$$

In [ ]:
N = 1000 # number of trials
T = 100 # recording time  
lam = 0.1 # rate


trials = []

# record spike times for N trials
for i in xrange(N):

    t = np.random.exponential(1./lam) # occurance of first spike
    
    spike_times = []

    while t < T: # record spike times until t > T
    
        spike_times.append(t)
        t += np.random.exponential(1./lam) 

    trials.append(spike_times)

# plot spike trains and number of spikes in each trial
fig = plt.figure()
ax0 = fig.add_subplot(1, 2, 1)  

spike_counts = np.zeros(N)

for i, trial in enumerate(trials):

    ax0.plot(trial, i*np.ones_like(trial), 'o', color = 'b', markersize = 0.5)
    spike_counts[i] = np.sum(trial)
    
ax0.set_xlabel('t')
ax0.set_ylabel('trial')

#ax1 = fig.add_subplot(1, 2, 2)

#for i, trial in enumerate(trials):

#    ax1.barh(np.arange(N), spike_counts, color = 'b', ecolor = 'k')

plt.show()